#  测试 llama.cpp http接口


In [ ]:
import json;
import urllib.request;


def chat_model(prompt, model="qwen3:latest", url="http://192.168.9.179:11434/api/chat"):
    data = {
         "model": model,
        "message": [
            {
                "role": "user", 
                "content": prompt
            }
        ],
        "options": {
            "seed": 123,
            "temperature": 0.0
        }
    };

    payload = json.dumps(data).encode("utf-8");
    request = urllib.request.Request(url, method="POST", data=payload);
    request.add_header("Content-Type", "application/json");
    content = "";
    with urllib.request.urlopen(request) as response:
        while True:
            line = response.readline().decode('utf-8');
            if not line:
                break;

            response_json = json.loads(line);

            content +=response_json["message"]["content"];
        return content;



In [19]:
request = chat_model("您好， 你是？？"
#, url="http://127.0.0.1:8899/v1/chat/completions"
);

printf(f"request:{request}");

URLError: <urlopen error [WinError 10061] No connection could be made because the target machine actively refused it>